[![Open Rendered Output](https://img.shields.io/badge/Rendered%20Output-Open-blue?logo=link&logoColor=white)](https://htmlpreview.github.io/?https://raw.githubusercontent.com/MeteoSwiss/nwp-fdb-polytope-demo/main/examples/snapshots/feature_bounding_box.html)

# Bounding Box Feature: Bounding Box around Switzerland

The following notebook demonstrates the full workflow to access, process and visualize ICON-CH1-EPS model data in a bounding box around Switzerland using 2-meter temperature data. 

<div style="text-align:center;">
  <img src="https://raw.githubusercontent.com/MeteoSwiss/nwp-fdb-polytope-demo/main/examples/Polytope/images/t_2m_bounding_box.png" style="width:50%;"/>
</div>

The data is retrieved using [Polytope](https://polytope.readthedocs.io/en/latest/), a feature extraction software developed by ECMWF. It applies concepts of computational geometry to extract n-dimensional polygons (also known as polytopes) from datacubes, such as a bounding box. To access MeteoSwiss' operational ICON-CH1-EPS and ICON-CH2-EPS model data, [meteodata-lab](https://meteoswiss.github.io/meteodata-lab/) provides a wrapper around the Polytope client that simplifies the request API. Follow the instructions to learn more about model data access via Polytope.

## Installation
Follow the instructions in [README.md](https://github.com/MeteoSwiss/nwp-fdb-polytope-demo/blob/main/README.md#Installation-1) to install the necessary dependencies.

## Configuring Access to Polytope
To access ICON data via MeteoSwiss's Polytope, you need a Polytope offline token provided by MeteoSwiss. If you do not already have a token, you can request one [here](https://meteoswiss.atlassian.net/wiki/spaces/IW2/pages/327780397/Polytope#Offline-token-authentication). Then, create a new `config.yml` file based on [`config_example.yml`](config_example.yml), and replace <meteoswiss_key> with your access token there. 

In [ ]:
import os
import yaml

def load_config(path="config.yml"):
    if not os.path.exists(path):
        raise FileNotFoundError("Missing config.yml. Please create one based on config_example.yml")
    with open(path, "r") as f:
        return yaml.safe_load(f)

config = load_config()

#ICON-CSCS Polytope credentials
os.environ["POLYTOPE_USER_KEY"] = config["meteoswiss"]["key"]
os.environ["POLYTOPE_ADDRESS"] = "https://polytope-depl.mchml.cscs.ch"

## Selecting date and time of the forecast

The FDB containing real-time data typically **holds only the most recent day of forecasts**. Therefore, it is necessary to specify the current date and select a corresponding forecast time in the past.

In [ ]:
from datetime import datetime, timedelta

# Current time
now = datetime.now()

# Subtract 12 hours
past_time = now - timedelta(hours=12)

# Round down to the nearest multiple of 6
rounded_hour = (past_time.hour // 6) * 6
rounded_time = past_time.replace(hour=rounded_hour, minute=0, second=0, microsecond=0)

# Format as YYYYMMDD and HHMM
date = rounded_time.strftime('%Y%m%d')
time = rounded_time.strftime('%H%M')
date,time

## Define the bounding box
In our example we are looking for a bounding box covering Switzerland. Therefore, we need to select two points. The first corresponding to the upper left and the second matching the bottom right corner of the bounding box.

In [ ]:
# point 1: top left corner, point 2: bottom right corner
geo_points = [[5.8, 47.81], [10.5, 45.81]] # bounding box around Switzerland

## Rotate the bounding box points

Given that the data source accessed by Polytope is stored on a rotated grid, it is necessary to provide Polytope with the bounding box in rotated coordinates, using a South Pole rotation with a reference of longitude 10° and latitude of -43°.
> **NOTE**: The function `transform_point()` expects first longitude and then latitude.

In [ ]:
import cartopy.crs as ccrs

# South pole rotation of lon=10, latitude=-43
rotated_crs = ccrs.RotatedPole(
    pole_longitude=190, pole_latitude=43
)

# Convert a point from geographic to rotated coordinates
geo_crs = ccrs.PlateCarree()
rotated_points = [
    rotated_crs.transform_point(lon, lat, geo_crs)
    for lon, lat in geo_points
]

## Define the request

Once the data is rotated, we need to define a MARS request using [meteodata-lab](https://polytope.readthedocs.io/en/latest/). The `feature` attribute allows you to extract **only the relevant data at the given points**. Thus, the amount of data that is retrieved from storage is significantly reduced. For the "bounding box" `feature` the following dictionary is needed.
> **NOTE**: Don't forget to specify that the data points are tuples that first contain longitude and then latitude.

In [ ]:
feature={
    "type" : "boundingbox",
    "points" : rotated_points,
    "axes" : ["longitude", "latitude"] #first longitude, then latitude
}

Finally, we can define the request. This example fetches **2-m temperature** from **ICON-CH1-EPS** at the **surface**, for the **control forecast** (`type="cf"`), at the selected run date/time.

In [ ]:
from meteodatalab import mars

request = mars.Request(
    param="T_2M",
    date=date,
    time=time,
    model=mars.Model.ICON_CH1_EPS,
    levtype=mars.LevType.SURFACE,
    type="cf",
    step=0,
    feature=feature
)

## Data retrieval
Now we use [earthkit.data](https://earthkit-data.readthedocs.io/en/latest/) to load the data and convert it into an [xarray.Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html).

In [ ]:
import earthkit.data as ekd
ds = ekd.from_source(
    "polytope",
    "mchgj",
    request.to_polytope(),
    stream=False
).to_xarray()

## Reverse rotation
The data in the dataset contains rotated longitudes and latitudes. To plot it, we will reverse the rotation.

In [ ]:
unrotated_points = [
    geo_crs.transform_point(lon, lat, rotated_crs)
    for lon, lat in zip(ds.longitude,ds.latitude)
]

geo_lons, geo_lats = zip(*unrotated_points)

ds_geo = ds.assign_coords(
    longitude=("points", list(geo_lons)),
    latitude=("points", list(geo_lats))
)

## Plotting

We use the library [earthkit.plots](https://earthkit-plots.readthedocs.io/en/latest/) to plot the data. Moreover, we can specify a styling with [earthkit.plots.styles](https://earthkit-plots.readthedocs.io/en/latest/examples/guide/04-styles.html#4.-Styles) to show a color spectrum from red to blue and convert the unit to Celsius for better readability.

In [ ]:
from earthkit.plots import Map
from earthkit.plots.styles import Style
from earthkit.plots.geo import bounds, domains

xmin, xmax = 5, 11   # Longitude bounds
ymin, ymax = 45.5, 48   # Latitude bounds

bbox = bounds.BoundingBox(xmin, xmax, ymin, ymax, ccrs.Geodetic())
domain = domains.Domain.from_bbox(
    bbox=bbox,
    name="CH2"
)

style = Style(colors="RdBu_r", units="celsius")
chart = Map(domain=domain)
chart.point_cloud(ds_geo["t_2m"], x="longitude", y="latitude", style=style)

chart.coastlines()
chart.borders()
chart.gridlines()


chart.title("Bounding Box: 2-Meter Temperature at {datetimes}")

chart.legend()

chart.show()